# Notes
make sure you are running on the right kernel \
start with uploading the data file to data assets. take the code from that location.


In [3]:
# !pip install azure-ai-ml --upgrade
# restart the kernel after this

In [4]:
# if you are getting the ModuleNotFoundError, run the following two commands. This is a kernel mismatch problem.
!which python

/anaconda/envs/azureml_py38/bin//python


In [12]:
!pip show azure-ai-ml


Name: azure-ai-ml
Version: 1.26.2
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 


## Load the Data

In [14]:
# copy this code from Data Assets

import pandas as pd
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())
data_asset = ml_client.data.get("titanicData", version="1")

df = pd.read_csv(data_asset.path)
df

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


,Unnamed: 0,Survived,Pclass,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare,Sex_female,Sex_male
0,116,0,2,29.0,1,0,21.0000,0,1
1,94,0,3,22.0,0,0,8.0500,0,1
2,864,0,3,23.0,0,0,9.5000,0,1
3,131,0,3,47.0,1,0,14.5000,1,0
4,95,0,1,71.0,0,0,34.6542,0,1
...,...,...,...,...,...,...,...,...,...
704,82,0,1,28.0,0,0,47.1000,0,1
705,752,0,3,28.0,0,0,7.7958,0,1
706,729,0,2,23.0,0,0,13.0000,0,1
707,286,1,2,42.0,0,0,13.0000,0,1


In [15]:
# Step 1: Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [16]:
# Step 3: Preprocess the data
# Select features and target
features = ['Pclass', 'Sex_female', 'Sex_male', 'Age', 'Fare']
df = df[features + ['Survived']].dropna()

In [17]:
# Encode categorical variable
# df['Sex'] = LabelEncoder().fit_transform(df['Sex'])  # Male = 1, Female = 0

# Step 4: Split data into train and test
X = df[features]
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train a model
model = LogisticRegression()
model.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2f}")

Accuracy: 0.82


In [6]:
import pickle

# Save the model
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)


## Loading a Model for Inferencing

In [8]:
# Load the model from a .pkl file
with open('model.pkl', 'rb') as f:
    titanic = pickle.load(f)


In [9]:
# Example input
input_data = pd.DataFrame([{
    'Pclass': 2,
    'Sex_female': 0,
    'Sex_male': 1,
    'Age': 30,
    'Fare': 20.5
}])

In [10]:
prediction = titanic.predict(input_data)
print(f"Prediction: {prediction[0]}")

Prediction: 0


In [11]:
prob = titanic.predict_proba(input_data)
print(f"Survival probability: {prob[0][1]:.2f}")

Survival probability: 0.26


### Creating Data for inference

In [20]:
import pandas as pd
from io import StringIO

data = """Pclass,Sex_female,Sex_male,Age,Fare
1,0,1,22,71.2833
1,1,0,38,71.2833
1,0,1,26,7.925
1,1,0,35,53.1
2,1,0,27,8.0500
2,0,1,30,12.4750
2,1,0,28,9.0000
3,0,1,19,13.0000
3,1,0,45,7.2292
3,0,1,40,6.8583
"""

# Convert string data into a pandas dataframe
df = pd.read_csv(StringIO(data))

# Save to CSV
df.to_csv("titanic_input_data.csv", index=False)


In [21]:
# Saving the input data

from azure.ai.ml.entities import Data

data_asset = Data(
    path="./titanic_input_data.csv",  # path to your local CSV file
    name="titanic_input_data",
    type="uri_file"
)

# Upload to Azure ML and register it
ml_client.data.create_or_update(data_asset)


Uploading titanic_input_data.csv (< 1 MB): 100%|██████████| 188/188 [00:00<00:00, 10.9kB/s]




Data({'path': 'azureml://subscriptions/265865a0-bc8c-4a43-bb85-77f02af85af5/resourcegroups/resourcegroup_1/workspaces/workspace_1/datastores/workspaceblobstore/paths/LocalUpload/12423bbad10a04a10926be354ff16060/titanic_input_data.csv', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_file', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'titanic_input_data', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/265865a0-bc8c-4a43-bb85-77f02af85af5/resourceGroups/resourcegroup_1/providers/Microsoft.MachineLearningServices/workspaces/workspace_1/data/titanic_input_data/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/cem1/code/Users/cem', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x751d27d71ed0>, 'serialize': <msrest.serialization.Serializer object at 0x751d27f8e9e0>, 'v

## Registering the model

In [13]:
from azure.ai.ml.entities import Model

model = Model(
    path="model.pkl",
    name="titanic-model",
    description="Logistic regression model for Titanic survival",
    type="custom_model",  # or "custom_model" or "mlflow_model"
)

# Register the model
registered_model = ml_client.models.create_or_update(model)
print(f"Model '{registered_model.name}' version {registered_model.version} registered.")


Model 'titanic-model' version 1 registered.


## Define a Command Job in the Pipeline

In [18]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import Data, Model
from azure.ai.ml import Input, Output
from azure.ai.ml.entities import CommandComponent

In [19]:
compute_list = ml_client.compute.list()

for compute in compute_list:
    print(compute.name)

cem1


In [12]:
!pip install --upgrade azure-ai-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 95.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: azure-ai-ml
    Found existing installation: azure-ai-ml 1.26.2
    Uninstalling azure-ai-ml-1.26.2:
      Successfully uninstalled azure-ai-ml-1.26.2


In [20]:
from azure.ai.ml.entities import Environment, PythonSection


# Create the environment object
env = Environment(
    name="custom-pip-environment2",
    description="Custom environment using pip requirements.txt",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    python=PythonSection(
        interpreter_path="python",
        user_managed_dependencies=False,
        requirements_txt="requirements.txt"
    )
)

# Register the environment
ml_client.environments.create_or_update(env)


ImportError: cannot import name 'PythonSection' from 'azure.ai.ml.entities' (/anaconda/envs/azureml_py38/lib/python3.10/site-packages/azure/ai/ml/entities/__init__.py)

In [26]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output

# Retrieve the registered model (not a data asset!)
model = ml_client.models.get(name="titanic-model", version="1")

job = command(
    code="./",  # folder containing score.py
    command="python score.py --input_data ${{inputs.input_data}} --model_path ${{inputs.model}} --output_data ${{outputs.output_data}}",
    inputs={
        "input_data": Input(type="uri_file", path="azureml:titanic_input_data:1"),  # <-- fix here
        "model": model.path,  
    },
    outputs={
        "output_data": Output(type="uri_file", mode="rw_mount")
    },
    environment="azureml:AzureML-sklearn-1.0-ubuntu20.04-py38-cpu:1",  # your custom env name and version
    compute="cem1"
)



In [27]:
returned_job = ml_client.jobs.create_or_update(job)
ml_client.jobs.stream(returned_job.name)


Uploading cem (0.08 MBs): 100%|██████████| 76830/76830 [00:00<00:00, 426310.29it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFileJobOutput'> and will be ignored


RunId: lucid_lock_xm90ptb4r5
Web View: https://ml.azure.com/runs/lucid_lock_xm90ptb4r5?wsid=/subscriptions/265865a0-bc8c-4a43-bb85-77f02af85af5/resourcegroups/resourcegroup_1/workspaces/workspace_1

Execution Summary
RunId: lucid_lock_xm90ptb4r5
Web View: https://ml.azure.com/runs/lucid_lock_xm90ptb4r5?wsid=/subscriptions/265865a0-bc8c-4a43-bb85-77f02af85af5/resourcegroups/resourcegroup_1/workspaces/workspace_1

Warnings:
Failed to execute command group with error An unexpected error occurred while executing command due to: Required environment variable not found: AZ_LS_CERT_THUMBPRINT. (inner_err: None)



JobException: Exception : 
 {
    "error": {
        "code": "ServiceError",
        "message": "Failed to execute command group with error An unexpected error occurred while executing command due to: Required environment variable not found: AZ_LS_CERT_THUMBPRINT. (inner_err: None)",
        "message_parameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z",
    "component_name": "CommonRuntime"
} 